In [55]:
import time
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, leaf_m = 3, leaf_n = 0):

    gain, t, index = find_best_split(data, labels)
    
    leaf_i = leaf_n

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    
    leaf_i += 1
    
    if leaf_i <= leaf_m:
        continue
        
    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, leaf_m, leaf_i)

#     print(time.time(), true_branch)
    false_branch = build_tree(false_data, false_labels, leaf_m, leaf_i)
    
#     print(time.time(), false_branch)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch, leaf_m, leaf_i)

SyntaxError: 'continue' not properly in loop (<ipython-input-55-5bbe7249606a>, line 19)